# Explore ERDDAP timeseries data
This notebook uses `Holoviz` tools to interactively explore the last two weeks of time series data from an ERDDAP Server. Select a `variable` from the list, then specify a station to see the time series.  

In [ ]:
from erddap_app.layout import layout

In [ ]:
server_name = "neracoos"

In [ ]:
progressbar

In [ ]:
valid_stdnames, server, e = get_valid_stdnames(server_name)

In [ ]:
trange = np.arange(
    datetime.datetime(2011, 1, 1), datetime.datetime.today(), datetime.timedelta(days=1)
)

wstdname_menu = pn.widgets.Select(
    name="Choose a variable",
    options=valid_stdnames,
)

dmap = hv.DynamicMap(replot_dsmap, kdims=["Stdnames", "TimeRange"])
dmap = dmap.redim.values(Stdnames=valid_stdnames, TimeRange=trange)

dmap = hv.DynamicMap(
    pn.bind(
        replot_dsmap,
        stdname=wstdname_menu,
        timerange=wstdname_date,
    )
)

In [ ]:
df = get_datasets(
    e,
    wstdname_menu.value,
    server.get("cdm_data_type"),
    wstdname_date.value_start,
    wstdname_date.value_end,
    server.get("skip_datasets"),
)
dsnames = list(df.datasetID.values)


wds_name = pn.widgets.Select(
    name="Choose a dataset",
    options=dsnames,
    value=dsnames[0],
)


def replot_tseries(dataset, timerange, stdname):

    constraints = {"time>=": timerange[0], "time<=": timerange[1]}

    df_tseries, var_tseries, unit_tseries = get_timeseries(
        e=e,
        dataset=dataset,  # from dswidgets
        stdname=wstdname_menu.value,  # from stdwidgets
        constraints=constraints,  # from dswidgets
    )

    # determine ylabel
    ylabel = df_tseries.columns[0]
    ylabel.replace("_", " ")

    # determine title
    dsname = dataset.replace("_", " ")
    varname = df_tseries.columns[0].replace("_", " ")

    # plot
    tseries = df_tseries.hvplot(
        kind="line",
        ylabel=unit_tseries,
        title=f"{dsname}   -   {varname}",
        grid=True,
        xticks=8,
        xformatter=formatter,
    )
    return tseries


dplot = hv.DynamicMap(replot_tseries, kdims=["Dataset", "TimeRange", "Stdnames"])
dplot = dplot.redim.values(
    Dataset=dsnames,
    TimeRange=trange,
    Stdnames=valid_stdnames,
)

dplot.opts(width=700, framewise=True)

dplot = hv.DynamicMap(
    pn.bind(
        replot_tseries,
        dataset=wds_name,
        timerange=wds_date,
        stdname=wstdname_menu,
    )
)

In [ ]:
def update_wds_name(event):
    df = get_datasets(
        e,
        wstdname_menu.value,
        server.get("cdm_data_type"),
        wstdname_date.value_start,
        wstdname_date.value_end,
        server.get("skip_datasets"),
    )
    dsnames = list(df.datasetID.values)
    wds_name.options = dsnames


wstdname_menu.param.watch(update_wds_name, "value")

In [ ]:
def update_wds_date(event):
    wds_date.value = wstdname_date.value


wstdname_date.param.watch(update_wds_date, "value")

In [ ]:
gspec = pn.GridSpec(sizing_mode="stretch_both", max_height=800)

gspec[0, 0] = dmap
gspec[0, 1] = pn.WidgetBox(
    "## ERDDAP timeseries explorer",
    wstdname_menu,
    wstdname_date,
    wds_name,
    wds_date,
)
gspec[1, :] = dplot

gspec